# 🎯 Explainability-Guided Fine-Tuning - Generalized

This notebook implements advanced fine-tuning guided by explainability insights.

**Configuration-driven approach:** All settings loaded from `../config/pipeline_config.json`

In [ ]:
# Import configuration system and utilities
import sys
import os
sys.path.append("../")

from src.pipeline_utils import ConfigManager, StateManager, LoggingManager
import torch
import numpy as np
from pathlib import Path
from datetime import datetime

# Initialize managers
config = ConfigManager("../config/pipeline_config.json")
state = StateManager("../config/pipeline_state.json")
logger = LoggingManager().get_logger("pipeline")

print("📋 Configuration loaded from ../config/pipeline_config.json")